# Notebook

## import packages

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow import keras
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MeanSquaredError
from keras_tuner.tuners import Hyperband
from sklearn.model_selection import train_test_split
from tensorflow.keras.regularizers import l2

from scipy.stats import zscore
import re
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

## data preprocessing

In [ ]:
def merge_data(GSVA_Score,modelID,fingerprint,IC50):
    #step1: merge score with modelID
    GSVA_Score_modelID=GSVA_Score.merge(
    modelID,
    how='inner',
    left_on='ccl_name', 
    right_on='ModelID' 
    )
    #step2: merge ic50 with fingerprint
    IC50['DRUG_ID']=IC50['DRUG_ID'].astype(str)
    fingerprint['Name']=fingerprint['Name'].astype(str)
    ic50_fingerprint=IC50.merge(
    fingerprint,
    how='inner',
    left_on='DRUG_ID',
    right_on='Name'
    )
    #step3: merge expression and ic50
    ic50_fingerprint['stripped_name']=[re.sub('\\-','',m) for m in ic50_fingerprint.CELL_LINE_NAME]
    data=ic50_fingerprint.merge(
        GSVA_Score_modelID,
        how='inner',
        left_on='stripped_name',
        right_on= 'StrippedCellLineName'
    )
    return data

+ GDSC1

In [ ]:
modelID=pd.read_csv("./dataset/Model.csv").iloc[:,[0,3]]
gdsc1_fingerprint=pd.read_csv("./dataset/all_molecule_morgan.csv").rename(columns={'Unnamed: 0':'Name'})
gdsc1_IC50=pd.read_csv("./dataset/GDSC1_selected.csv")
gdsc1_data_df=merge_data(reduced_data_df,modelID,gdsc1_fingerprint,gdsc1_IC50)

gdsc1_X=gdsc1_data_df.drop(['CELL_LINE_NAME','DRUG_NAME','DRUG_ID','Name','ModelID','StrippedCellLineName','LN_IC50','stripped_name','ccl_name'],axis=1)
gdsc1_y=gdsc1_data_df['LN_IC50']

+ GDSC2

In [ ]:
modelID=pd.read_csv("./dataset/Model.csv").iloc[:,[0,3]]
gdsc2_fingerprint=pd.read_csv("./dataset/all_molecule_morgan.csv").rename(columns={'Unnamed: 0':'Name'})
gdsc2_IC50=pd.read_csv("./dataset/GDSC2_selected.csv")
gdsc2_data_df=merge_data(reduced_data_df,modelID,gdsc2_fingerprint,gdsc2_IC50)

gdsc2_X=gdsc2_data_df.drop(['CELL_LINE_NAME','DRUG_NAME','DRUG_ID','Name','ModelID','StrippedCellLineName','LN_IC50','stripped_name','ccl_name'],axis=1)
gdsc2_y=gdsc2_data_df['LN_IC50']

In [ ]:
gdsc1_data_df.to_csv("gdsc1_data_for_model_building.csv")
gdsc2_data_df.to_csv("gdsc2_data_for_model_building.csv")

In [ ]:
X=pd.concat([gdsc1_X,gdsc2_X])
y=pd.concat([gdsc1_y,gdsc2_y])
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=1024)

## autoencoder modeling

In [ ]:
expression_CCLE=pd.read_csv("./dataset/OmicsExpressionProteinCodingGenesTPMLogp1.csv")
expression_CCLE.columns=[re.sub("\\s*\\(.*","",m) for m in expression_CCLE.columns]
expression_CCLE=expression_CCLE.rename(columns={'Unnamed: 0':'ccl_name'})

In [ ]:
gene_expression=expression_CCLE.drop(['ccl_name'],axis=1).apply(zscore,axis=0)
input_dim = gene_expression.shape[1]  
encoding_dim = 30  


input_layer = keras.Input(shape=(input_dim,))
# encoded layers
encoded = Dense(2048, activation='elu')(input_layer)
encoded = Dense(encoding_dim, activation='relu')(encoded)
# decode layers
decoded = Dense(2048, activation='elu')(encoded)
decoded = Dense(input_dim, activation='relu')(decoded)
# modeling
autoencoder = Model(input_layer, decoded)
encoder = Model(input_layer, encoded)
autoencoder.compile(optimizer='adam', loss='mse')
early_stopping = callbacks.EarlyStopping(
    min_delta=0.001, # minimium amount of change to count as an improvement
    patience=10, # how many epochs to wait before stopping
    restore_best_weights=True,
)
# building
autoencoder.fit(gene_expression, gene_expression, epochs=50, 
                batch_size=256, shuffle=True, validation_split=0.2, callbacks=[early_stopping],)

#reduction
reduced_data = encoder.predict(gene_expression)
# saved as DataFrame
reduced_data_df = pd.DataFrame(reduced_data, columns=[f'encoded_{i+1}' for i in range(encoding_dim)])
reduced_data_df['ccl_name']=expression_CCLE['ccl_name']
# model save
# autoencoder.save('autoencoder_model.h5')

## durgs modeling

In [ ]:
model = keras.Sequential()
model.add(keras.Input(shape=(X_train.shape[1],)))
model.add(layers.Dense(1024, activation="relu"))
model.add(layers.Dropout(0.1))
model.add(layers.Dense(16, activation="relu"))
model.add(layers.Dropout(0.1))
model.add(layers.Dense(1))

def custom_loss(y_true, y_pred):
    loss = tf.reduce_mean(tf.square(y_true - y_pred))
    return loss
optimizer = Adam(learning_rate=1e-3, clipvalue=1.0)
model.compile(optimizer=Adam(learning_rate=0.00011),
              #optimizer=optimizer,
              loss=MeanSquaredError(),
              #loss=custom_loss,
              metrics=['mae'])

early_stopping = callbacks.EarlyStopping(
    min_delta=0.001, # minimium amount of change to count as an improvement
    patience=15, # how many epochs to wait before stopping
    restore_best_weights=True,
)
model.summary()

In [ ]:
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    batch_size=128,
    epochs=200,
    callbacks=[early_stopping], # put your callbacks in a list
    verbose=1,  # turn off training log
)

## predictions

+ GDSC1

In [ ]:
tmp=model.predict(gdsc1_X)
results=pd.DataFrame({
    "y_valid":gdsc1_y,
    "predictions":[m[0] for m in tmp]
})
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(10, 6))
sns.scatterplot(x='y_valid', y='predictions', data=results)
plt.title('GDSC1')
plt.xlabel('y_valid')
plt.ylabel('predictions')
correlation = results['y_valid'].corr(results['predictions'])
plt.annotate(f'Correlation: {correlation:.2f}', xy=(0.05, 0.95), xycoords='axes fraction', fontsize=12, bbox=dict(facecolor='white', alpha=0.6))
plt.show()

+ GDSC2

In [ ]:
tmp=model.predict(gdsc2_X)
results=pd.DataFrame({
    "y_valid":gdsc2_y,
    "predictions":[m[0] for m in tmp]
})

plt.figure(figsize=(10, 6))
sns.scatterplot(x='y_valid', y='predictions', data=results)
plt.title('GDSC2')
plt.xlabel('y_valid')
plt.ylabel('predictions')
correlation = results['y_valid'].corr(results['predictions'])
plt.annotate(f'Correlation: {correlation:.2f}', xy=(0.05, 0.95), xycoords='axes fraction', fontsize=12, bbox=dict(facecolor='white', alpha=0.6))
plt.show()

## applicaitons

### predictions for GSE151343

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import zscore
from tensorflow.keras.regularizers import l2
def transform_data(gene_matrix_files_in):
    sc_count=pd.read_csv(gene_matrix_files_in,index_col=0).T
    features=np.load('./dataset/features.npy', allow_pickle=True)
    result_df = sc_count.reindex(columns=expression_CCLE.columns, fill_value=0)
    #zscore
    result_df=result_df.apply(zscore,axis=0)
    #na FILL
    result_df = result_df.fillna(0)
    return result_df

In [ ]:
def reduction_dimension(result_df):
    # random seed
    tf.random.set_seed(42)
    np.random.seed(42)
    # input layers
    input_dim=19193 #gene numbers
    input_layer = keras.Input(shape=(input_dim,))
    # encoding layers
    encoded = Dense(2048, activation='elu')(input_layer)
    encoded = Dense(encoding_dim, activation='relu')(encoded)
    # decoding layers
    decoded = Dense(2048, activation='elu')(encoded)
    decoded = Dense(input_dim, activation='relu')(decoded)
    autoencoder = Model(input_layer, decoded)
    encoder = Model(input_layer, encoded)
    autoencoder.compile(optimizer='adam', loss='mse')
    early_stopping = callbacks.EarlyStopping(
        min_delta=0.001, # minimium amount of change to count as an improvement
        patience=10, # how many epochs to wait before stopping
        restore_best_weights=True,
    )
    autoencoder=keras.models.load_model("./autoencoder_model.h5") # not provided, you can train the model by yourself
    result_df_sc = encoder.predict(result_df)
    result_df_sc = pd.DataFrame(result_df_sc, columns=[f'encoded_{i+1}' for i in range(30)])
    return result_df_sc

def get_compound_predicted_ic50(fp):
    row_to_add_df = pd.DataFrame([fp] * len(result_df_sc)).reset_index(drop=True)
    result = pd.concat([row_to_add_df, result_df_sc], axis=1)
    tmp=model.predict(result)
    res=pd.Series([m[0] for m in tmp])
    return res

In [ ]:
result_df=transform_data('GSE151343_expression.csv')
result_df_sc=reduction_dimension(result_df) #reduction

In [ ]:
fp=pd.read_csv('./dataset/all_molecule_morgan.csv',index_col=0)
model=keras.models.load_model("./dnn_model_basedon_encoder30_gdsc1_gdsc2/")

In [ ]:
new_df=fp.apply(get_compound_predicted_ic50,axis=1)
new_df.columns=result_df.index
new_df

In [ ]:
new_df.to_csv("GSE151343_prediction.csv")

### predictions for TCGA

In [ ]:
result_df=transform_data('tcga_gene_expression.csv')
result_df_sc=reduction_dimension(result_df)
fp=pd.read_csv('./dataset/all_molecule_morgan.csv',index_col=0)
model=keras.models.load_model("./dnn_model_basedon_encoder30_gdsc1_gdsc2/")

In [ ]:
new_df=fp.apply(get_compound_predicted_ic50,axis=1)
new_df.columns=result_df.index
new_df

In [ ]:
new_df.to_csv("tcga_prediction.csv")